In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Gas price data

#### This notebooks combines Gas price data, Binance ETH/USD data, and block gaslimit/gas usage

#### Gas price data was retrieved from google big query, which provides a queryable database of Ethereum blockchaind data

#### Retrieved from the GCP BigQuery API using the following query;

In [ ]:
SELECT t0.avg_gas_price,

t0.block_timestamp,

t0.max_gas_price,

t0.min_gas_price 

FROM (SELECT
  block_timestamp, MIN(gas_price) as min_gas_price, MAX(gas_price) as max_gas_price, AVG(gas_price) as avg_gas_price
  
FROM 'bigquery-public-data.crypto_ethereum.transactions'

WHERE
  DATE(block_timestamp) between  "2021-11-26"  AND "2022-05-26"
  
GROUP BY block_timestamp) AS t0

ORDER BY block_timestamp DESC 

## Process dataset for later modelling 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
#Load Query CSV generate by above query
df = pd.read_csv ('/content/drive/MyDrive/Practicum/Github/data/Block Gas Prices 2021-11-26  to 2022-05-26/bq-results-20220512-120428-1652357092099.csv', header=0)

#Read datetime, sort by chronologcailly and index data by datetime
df['datetime'] = pd.to_datetime(df['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')
df['timestamp'] = df['datetime'].astype(np.int64) // 10**9
df = df.sort_values(by='block_timestamp',ascending=False)
df = df.set_index('datetime')

#Resampel data, taking mean over 1 minute window labelled with the tiem at the left side of the window
df_block_gas_price = df.resample('1T').mean()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


,avg_gas_price,max_gas_price,min_gas_price,timestamp
datetime,,,,
2022-05-12 11:56:00,NaN,NaN,NaN,NaN
2022-05-12 11:57:00,2.108981e+11,5.264396e+11,1.962703e+11,1.652357e+09
2022-05-12 11:58:00,2.267423e+11,8.738117e+11,2.083214e+11,1.652357e+09
2022-05-12 11:59:00,1.875731e+11,1.167135e+12,1.713128e+11,1.652357e+09
2022-05-12 12:00:00,1.834676e+11,2.779116e+11,1.755539e+11,1.652357e+09


In [ ]:
#Load ETH/USD data previously downloaded from binance and merged
df = pd.read_csv ('/content/drive/MyDrive/Practicum/Github/data/ETH/eth_price_data_2021-01-01_2022-04-30.csv')

#Read datetime, sort by chronologcailly and index data by datetime, unix timestamp will be used later for coherence plots
df['Open_time'] = df['Open_time'].floordiv(1000)
df['datetime'] = pd.to_datetime(df['Open_time'], unit='s')
df= df.sort_values(by='Open_time',ascending=False)

df = df.set_index('datetime')
df_eth = df[['Open_time', 'Open']]
df_eth.tail()

,Open_time,Open
datetime,,
2021-01-01 00:04:00,1609459440,737.38
2021-01-01 00:03:00,1609459380,737.74
2021-01-01 00:02:00,1609459320,738.78
2021-01-01 00:01:00,1609459260,737.12
2021-01-01 00:00:00,1609459200,736.42


## Block gas used, base gas fee, transaction count and block size also queried

In [ ]:
##Below query used to retrieve data from bigquery 

SELECT timestamp, gas_used, base_fee_per_gas, transaction_count, size
FROM
  `bigquery-public-data.crypto_ethereum.blocks`
WHERE
  DATE(timestamp) between  "2021-11-26"  AND "2022-05-26"

ORDER BY timestamp DESC 

In [ ]:
#Load data, convert account for datetime formatting, add unix timestamp, sort by datetime, resample
df = pd.read_csv ('/content/drive/MyDrive/Practicum/Github/data/Block gas used, size, transaction count, base fee 2021-11-26 to 2022-05-26/bq-results-20220512-140101-1652364073656.csv', header=0)
df['datetime'] = pd.to_datetime(df['timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')
df['timestamp'] = df['datetime'].astype(np.int64) // 10**9
df = df.sort_values(by='timestamp',ascending=False)
df = df.set_index('datetime')
df_block_gas_usage = df.resample('1T').mean()
df_block_gas_usage.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  after removing the cwd from sys.path.


,timestamp,gas_used,base_fee_per_gas,transaction_count,size
datetime,,,,,
2022-05-12 13:51:00,1.652363e+09,2.208073e+07,1.812527e+11,323.000,179015.666667
2022-05-12 13:52:00,1.652364e+09,1.135943e+07,1.697456e+11,155.125,78191.625000
2022-05-12 13:53:00,1.652364e+09,1.698965e+07,1.485967e+11,229.500,195478.333333
2022-05-12 13:54:00,1.652364e+09,1.395524e+07,1.509106e+11,228.750,106518.000000
2022-05-12 13:55:00,1.652364e+09,1.173212e+07,1.530920e+11,160.000,55386.500000


# Merging datasets

We will merge datsets for for simple access in later modelling

In [ ]:
#Merge block gas limit/usage with ETH/USD exchagne data
merged_eth_usage = df_block_gas_usage.merge(df_eth, left_index=True, right_index=True)

In [ ]:
merged_eth_usage.head()

,timestamp,gas_used,base_fee_per_gas,transaction_count,size,Open_time,Open
datetime,,,,,,,
2021-11-26 00:00:00,1.637885e+09,1.347163e+07,8.143553e+10,163.875,49614.000000,1637884800,4524.48
2021-11-26 00:01:00,1.637885e+09,1.516819e+07,8.186168e+10,177.000,56027.333333,1637884860,4529.90
2021-11-26 00:02:00,1.637885e+09,1.820355e+07,7.465829e+10,284.500,107881.000000,1637884920,4528.51
2021-11-26 00:03:00,1.637885e+09,1.802044e+07,8.328165e+10,214.000,65871.600000,1637884980,4533.54
2021-11-26 00:04:00,1.637885e+09,1.573136e+07,9.560271e+10,185.500,61995.000000,1637885040,4534.97


In [ ]:
#Merge Gas price data
df_merged = merged_eth_usage.merge(df_block_gas_price, left_index=True, right_index=True)
df_merged.head()

,timestamp_x,gas_used,base_fee_per_gas,transaction_count,size,Open_time,Open,avg_gas_price,max_gas_price,min_gas_price,timestamp_y
datetime,,,,,,,,,,,
2021-11-26 00:00:00,1.637885e+09,1.347163e+07,8.143553e+10,163.875,49614.000000,1637884800,4524.48,9.687954e+10,3.401478e+11,8.168553e+10,1.637885e+09
2021-11-26 00:01:00,1.637885e+09,1.516819e+07,8.186168e+10,177.000,56027.333333,1637884860,4529.90,1.005555e+11,3.918277e+11,8.240792e+10,1.637885e+09
2021-11-26 00:02:00,1.637885e+09,1.820355e+07,7.465829e+10,284.500,107881.000000,1637884920,4528.51,9.189840e+10,4.351021e+11,7.540829e+10,1.637885e+09
2021-11-26 00:03:00,1.637885e+09,1.802044e+07,8.328165e+10,214.000,65871.600000,1637884980,4533.54,9.104711e+10,3.787140e+11,8.379399e+10,1.637885e+09
2021-11-26 00:04:00,1.637885e+09,1.573136e+07,9.560271e+10,185.500,61995.000000,1637885040,4534.97,1.081414e+11,5.355103e+11,9.635271e+10,1.637885e+09


In [ ]:
#Convert gas price data from wei to gwei, to giga-wei, or gwei, the most commonly used ETH denominator when referencing gas price 
df_merged_gwei = df_merged
df_merged_gwei['min_gas_price'] = df_merged['min_gas_price'].apply(lambda x: x/1000000000)
df_merged_gwei['avg_gas_price'] = df_merged['avg_gas_price'].apply(lambda x: x/1000000000)
df_merged_gwei['max_gas_price'] = df_merged['max_gas_price'].apply(lambda x: x/1000000000)
df_merged_gwei['base_fee_per_gas'] = df_merged['base_fee_per_gas'].apply(lambda x: x/1000000000)

#Get proportion of allocated block gas that was actually used

df_merged_gwei = df_merged_gwei.drop(['timestamp_x', 'Open_time', 'timestamp_y'], axis=1)
df_merged_gwei.head()


,gas_used,base_fee_per_gas,transaction_count,size,Open,avg_gas_price,max_gas_price,min_gas_price
datetime,,,,,,,,
2021-11-26 00:00:00,1.347163e+07,81.435534,163.875,49614.000000,4524.48,96.879536,340.147753,81.685534
2021-11-26 00:01:00,1.516819e+07,81.861680,177.000,56027.333333,4529.90,100.555493,391.827702,82.407921
2021-11-26 00:02:00,1.820355e+07,74.658292,284.500,107881.000000,4528.51,91.898398,435.102050,75.408292
2021-11-26 00:03:00,1.802044e+07,83.281655,214.000,65871.600000,4533.54,91.047115,378.713985,83.793994
2021-11-26 00:04:00,1.573136e+07,95.602713,185.500,61995.000000,4534.97,108.141411,535.510339,96.352713


## Save merged data at different sampling rates

In [ ]:
df_merged_gwei.head(20)

,gas_used,base_fee_per_gas,transaction_count,size,Open,avg_gas_price,max_gas_price,min_gas_price
datetime,,,,,,,,
2021-11-26 00:00:00,1.347163e+07,81.435534,163.875000,49614.000000,4524.48,96.879536,340.147753,81.685534
2021-11-26 00:01:00,1.516819e+07,81.861680,177.000000,56027.333333,4529.90,100.555493,391.827702,82.407921
2021-11-26 00:02:00,1.820355e+07,74.658292,284.500000,107881.000000,4528.51,91.898398,435.102050,75.408292
2021-11-26 00:03:00,1.802044e+07,83.281655,214.000000,65871.600000,4533.54,91.047115,378.713985,83.793994
2021-11-26 00:04:00,1.573136e+07,95.602713,185.500000,61995.000000,4534.97,108.141411,535.510339,96.352713
2021-11-26 00:05:00,1.983507e+07,95.494792,255.500000,93293.500000,4535.31,108.761250,343.886174,96.119792
2021-11-26 00:06:00,1.372897e+07,103.423644,234.000000,56190.333333,4528.10,109.227136,177.356359,103.423644
2021-11-26 00:07:00,1.615421e+07,93.772434,208.750000,69343.500000,4524.47,104.019914,476.668863,94.259212
2021-11-26 00:08:00,1.503235e+07,97.382915,206.750000,59445.000000,4530.10,114.162854,673.936760,95.587627


In [ ]:
df_merged_gwei.to_csv('/content/drive/MyDrive/Practicum/Github/data/ETH,gas,usage merged 11-26 to 05-26.csv', sep=',')

In [ ]:
df_merged_gwei_5min = df_merged_gwei.resample('5T').mean()
df_merged_gwei_60min = df_merged_gwei.resample('60T').mean()
df_merged_gwei_10min = df_merged_gwei.resample('10T').mean()

In [ ]:
df_merged_gwei_5min.to_csv(r'C:\Users\conal\Desktop\MCM\Practicum\data\ETH,gas,usage merged_5min 11-26 to 01-26.csv', sep=',')

In [ ]:
df_merged_gwei_60min.to_csv(r'C:\Users\conal\Desktop\MCM\Practicum\data\ETH,gas,usage merged_60min 11-26 to 01-26.csv', sep=',')

In [ ]:

df_merged_gwei_10min.to_csv(r'C:\Users\conal\Desktop\MCM\Practicum\data\ETH,gas,usage merged_10min 11-26 to 01-26.csv', sep=',')